In [1]:
#default_exp anime.kid

In [2]:
#export
from fast.torch_basics import *
from fast.metrics import *
from fast.learner import *
from torchvision.models.inception import Inception3
from sklearn.metrics.pairwise import polynomial_kernel

In [12]:
#export
#Reimplimentation to allow more hooks, only works for eval, not accurate for training
class Inception_v3(Inception3):
    def __init__(self):
        super(Inception_v3, self).__init__()
        self.pool3=nn.AdaptiveAvgPool2d((1, 1))
        #not using nn.Sequential as would double register parameters
        self.seq_modules=[self.Conv2d_1a_3x3,
            self.Conv2d_2a_3x3,
            self.Conv2d_2b_3x3,
            nn.MaxPool2d(3,2),
            self.Conv2d_3b_1x1,
            self.Conv2d_4a_3x3,
            nn.MaxPool2d(3,2),
            self.Mixed_5b,
            self.Mixed_5c,
            self.Mixed_5d,
            self.Mixed_6a,
            self.Mixed_6b,
            self.Mixed_6c,
            self.Mixed_6d,
            self.Mixed_6e,
            self.Mixed_7a,
            self.Mixed_7b,
            self.Mixed_7c,
            self.pool3,
            partial(F.dropout,training=False),
            nn.Flatten(1),
            self.fc]
    def forward(self, x):
        if self.transform_input:
            x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((x_ch0, x_ch1, x_ch2), 1)
        for mod in self.seq_modules:
            x = mod(x)
        return x

class KernelInceptionDistanceMetric(Metric):
    def __init__(self,targs):
        state = torch.hub.load('pytorch/vision:v0.4.2', 'inception_v3', pretrained=True).state_dict()
        inception=Inception_v3()
        inception.load_state_dict(state)
        inception.seq_modules=inception.seq_modules[0:-3] #removing layers up to pool3
        self.kid=KernelInceptionDistance(inception,targs)
        self.preds=[]
    def accumulate(self, learn):
        self.preds+=learn.pred.detach().cpu()
    def reset(self): 
        self.preds=[]
    @property
    def value(self):
        samples = self.kid(self.preds)
        means,stds = samples[:,0],samples[:,1]
        return means.mean(),means.std(),stds.mean(),stds.std()

class KIDCallback(Callback):
    def __init__(self,targs):
        self.kid=KernelInceptionDistanceMetric(targs)
    def batch_end(self):
        self.kid.accumulate(self.learn)
    def valid_end(self):
        self.kid.reset()

In [4]:
#export
class KernelInceptionDistance():
    # Images expected as [ b, channels, width, height 
    def __init__(self, model, images):
        self.model=model
        self.targs=array([self._inception_activations(imgs).numpy() for imgs in self._reshape_imgs(images)]).reshape(len(images),2048)
    
    def __call__(self,preds,subsets=100,subset_size=1000):
        preds=array([self._inception_activations(imgs).numpy() for imgs in self._reshape_imgs(preds)]).reshape(len(preds),2048) #[bs,2048] output expected from inception
        results=[]
        for i in range(subsets):
            t=np.random.choice(len(self.targs), subset_size, replace=False)
            p=np.random.choice(len(preds), subset_size, replace=False)
            results+=[self._polynomial_mmd(self.targs[t],preds[p])]
        return array(results)
    
    def _inception_activations(self,imgs):
        with torch.no_grad(): 
            return self.model(imgs).detach()[...,0,0]
    def _reshape_imgs(self,images,num=100):
        return images.reshape(len(images)//num,num,3,299,299)
    def _polynomial_mmd(self,codes_g, codes_r, degree=3, gamma=None, coef0=1,
                       var_at_m=None, ret_var=True):
        # use  k(x, y) = (gamma <x, y> + coef0)^degree
        # default gamma is 1 / dim
        X = codes_g
        Y = codes_r

        K_XX = polynomial_kernel(X, degree=degree, gamma=gamma, coef0=coef0)
        K_YY = polynomial_kernel(Y, degree=degree, gamma=gamma, coef0=coef0)
        K_XY = polynomial_kernel(X, Y, degree=degree, gamma=gamma, coef0=coef0)

        return self._mmd2_and_variance(K_XX, K_XY, K_YY,
            var_at_m=var_at_m, ret_var=ret_var)
    def _sqn(self,arr):
        flat = np.ravel(arr)
        return flat.dot(flat)
    def _mmd2_and_variance(self,K_XX, K_XY, K_YY, unit_diagonal=False,
                           mmd_est='unbiased', block_size=1024,
                           var_at_m=None, ret_var=True):
        # based on
        # https://github.com/dougalsutherland/opt-mmd/blob/master/two_sample/mmd.py
        # but changed to not compute the full kernel matrix at once
        m = K_XX.shape[0]
        assert K_XX.shape == (m, m)
        assert K_XY.shape == (m, m)
        assert K_YY.shape == (m, m)
        if var_at_m is None:
            var_at_m = m

        # Get the various sums of kernels that we'll use
        # Kts drop the diagonal, but we don't need to compute them explicitly
        if unit_diagonal:
            diag_X = diag_Y = 1
            sum_diag_X = sum_diag_Y = m
            sum_diag2_X = sum_diag2_Y = m
        else:
            diag_X = np.diagonal(K_XX)
            diag_Y = np.diagonal(K_YY)

            sum_diag_X = diag_X.sum()
            sum_diag_Y = diag_Y.sum()

            sum_diag2_X = self._sqn(diag_X)
            sum_diag2_Y = self._sqn(diag_Y)

        Kt_XX_sums = K_XX.sum(axis=1) - diag_X
        Kt_YY_sums = K_YY.sum(axis=1) - diag_Y
        K_XY_sums_0 = K_XY.sum(axis=0)
        K_XY_sums_1 = K_XY.sum(axis=1)

        Kt_XX_sum = Kt_XX_sums.sum()
        Kt_YY_sum = Kt_YY_sums.sum()
        K_XY_sum = K_XY_sums_0.sum()

        if mmd_est == 'biased':
            mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)
                    + (Kt_YY_sum + sum_diag_Y) / (m * m)
                    - 2 * K_XY_sum / (m * m))
        else:
            assert mmd_est in {'unbiased', 'u-statistic'}
            mmd2 = (Kt_XX_sum + Kt_YY_sum) / (m * (m-1))
            if mmd_est == 'unbiased':
                mmd2 -= 2 * K_XY_sum / (m * m)
            else:
                mmd2 -= 2 * (K_XY_sum - np.trace(K_XY)) / (m * (m-1))

        if not ret_var:
            return mmd2

        Kt_XX_2_sum = self._sqn(K_XX) - sum_diag2_X
        Kt_YY_2_sum = self._sqn(K_YY) - sum_diag2_Y
        K_XY_2_sum = self._sqn(K_XY)

        dot_XX_XY = Kt_XX_sums.dot(K_XY_sums_1)
        dot_YY_YX = Kt_YY_sums.dot(K_XY_sums_0)

        m1 = m - 1
        m2 = m - 2
        zeta1_est = (
            1 / (m * m1 * m2) * (
                self._sqn(Kt_XX_sums) - Kt_XX_2_sum + self._sqn(Kt_YY_sums) - Kt_YY_2_sum)
            - 1 / (m * m1)**2 * (Kt_XX_sum**2 + Kt_YY_sum**2)
            + 1 / (m * m * m1) * (
                self._sqn(K_XY_sums_1) + self._sqn(K_XY_sums_0) - 2 * K_XY_2_sum)
            - 2 / m**4 * K_XY_sum**2
            - 2 / (m * m * m1) * (dot_XX_XY + dot_YY_YX)
            + 2 / (m**3 * m1) * (Kt_XX_sum + Kt_YY_sum) * K_XY_sum
        )
        zeta2_est = (
            1 / (m * m1) * (Kt_XX_2_sum + Kt_YY_2_sum)
            - 1 / (m * m1)**2 * (Kt_XX_sum**2 + Kt_YY_sum**2)
            + 2 / (m * m) * K_XY_2_sum
            - 2 / m**4 * K_XY_sum**2
            - 4 / (m * m * m1) * (dot_XX_XY + dot_YY_YX)
            + 4 / (m**3 * m1) * (Kt_XX_sum + Kt_YY_sum) * K_XY_sum
        )
        var_est = (4 * (var_at_m - 2) / (var_at_m * (var_at_m - 1)) * zeta1_est
                   + 2 / (var_at_m * (var_at_m - 1)) * zeta2_est)

        return mmd2, var_est

In [5]:
state = torch.hub.load('pytorch/vision:v0.4.2', 'inception_v3', pretrained=True).state_dict()
inception=Inception_v3()
inception.load_state_dict(state)
inception.seq_modules=inception.seq_modules[0:-3] #removing layers up to pool3

Using cache found in /home/fast/.cache/torch/hub/pytorch_vision_v0.4.2


In [6]:
path=Path("/home/fast/.fastai/data/danbooru2018")

In [7]:
selfie_imgs=[]
with open(path/'selfie_mmd_test_full.npy', 'rb') as f:
   selfie_imgs=np.load(f)

In [8]:
anime_imgs=[]
with open(path/'anime_mmd_test_full.npy', 'rb') as f:
   anime_imgs=np.load(f)

In [9]:
kid = KernelInceptionDistance(inception,Tensor(anime_imgs.reshape(3400,3,299,299)))

In [11]:
kid(Tensor(selfie_imgs.reshape(3400,3,299,299)))

array([[-8.09724725e-05, -2.10227725e-08],
       [-2.33440941e-05, -2.10831747e-08],
       [-6.21251251e-05, -1.81319467e-08],
       [-8.30162663e-05, -2.24762155e-08],
       [-7.35260260e-05, -2.14923515e-08],
       [-9.67124625e-05, -1.99856415e-08],
       [-9.88083083e-05, -1.96475778e-08],
       [-1.08430180e-04, -2.26157290e-08],
       [-1.45734234e-04, -2.01962180e-08],
       [-1.13360110e-04, -2.09984027e-08],
       [-1.20378879e-04, -2.38314152e-08],
       [-1.29276777e-04, -2.26356037e-08],
       [-1.17204705e-04, -1.65683061e-08],
       [-1.10112112e-04, -2.04787056e-08],
       [-1.09458709e-04, -1.99636885e-08],
       [-1.26397898e-04, -2.40292361e-08],
       [-1.08704454e-04, -2.07743792e-08],
       [-9.35190190e-05, -2.19159081e-08],
       [-7.12567568e-05, -2.00620177e-08],
       [-9.05027528e-05, -2.05180839e-08],
       [-1.17563564e-04, -2.03607915e-08],
       [-8.44169169e-05, -1.65280018e-08],
       [-1.02600601e-04, -2.39787728e-08],
       [-7.

In [16]:
#hide
#from fast.notebook.export import notebook2script
#notebook2script(all_fs=True)